In [97]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint
import re

In [91]:
year = 2000
df = pd.read_csv("rt_data/rt_{}.csv".format(year))
link = df["RTLink"].tolist()[0]


In [102]:
rt_list = []
page = requests.get(link)
bs = BeautifulSoup(page.text)
scores = [elem.text.strip() for elem in bs.findAll("span", {"class": "mop-ratings-wrap__percentage"})]
if len(scores) == 2:
    tomato_score = scores[0]
    aud_score = scores[1]
else:
    raise Exception("Could not find all scores:\n{}".format(scores))
synopsis = bs.findAll("div", {"id": "movieSynopsis"})
if len(synopsis) == 1:
    synopsis = synopsis[0].text.strip()
else:
    raise Exception("Could not find all synopsis:\n{}".format(synopsis))
meta_data = bs.findAll("li", {"class": "meta-row clearfix"})
if len(meta_data) == 8:
    for data in meta_data:
        if "Rating: " in data.find("div", {"class": "meta-label subtle"}).text:
            rating = data.find("div", {"class": "meta-value"}).text.strip()
        if "Genre: " in data.find("div", {"class": "meta-label subtle"}).text:
            genres = [
                {
                    "link":"https://rottentomatoes.com" + link.get('href'),
                    "text":link.text
                } 
                for link in data.findAll("a")
            ]
        if "Directed By: " in data.find("div", {"class": "meta-label subtle"}).text:
            directors = [
                {
                    "link":"https://rottentomatoes.com" + link.get('href'),
                    "text":link.text
                } 
                for link in data.findAll("a")
            ]
        if "Written By: " in data.find("div", {"class": "meta-label subtle"}).text:
            writers = [
                {
                    "link":"https://rottentomatoes.com" + link.get('href'),
                    "text":link.text
                } 
                for link in data.findAll("a")
            ]
        if "In Theaters: " in data.find("div", {"class": "meta-label subtle"}).text:
            date_theaters = data.find("time")['datetime']
        if "On Disc/Streaming:" in data.find("div", {"class": "meta-label subtle"}).text:
            date_disc = data.find("time")['datetime']
        if "Runtime: " in data.find("div", {"class": "meta-label subtle"}).text:
            runtime = data.find("div", {"class": "meta-value"}).text.strip()
            runtime = re.findall(r"([0-9]+)\sminutes", runtime)
            if len(runtime) == 1:
                runtime = int(runtime[0])
            else:
                raise Exception("Could not parse runtime {}".format(data))
        if "Studio: " in data.find("div", {"class": "meta-label subtle"}).text:
            studio = data.find("div", {"class": "meta-value"}).text.strip()
else:
    raise Exception("Could not find all metadata:\n{}".format(meta_data))
rt_list.append(
    {
        "tomato_score":int(tomato_score[:-1]),
        "aud_score":int(aud_score[:-1]),
        "synopsis":synopsis,
        "rating":rating,
        "genres":genres,
        "directors":directors,
        "writers":writers,
        "date_theaters":date_theaters,
        "date_disc":date_disc,
        "runtime":runtime
    }
)


'2000-04-17T17:00:00-07:00'

In [109]:
csv_file = "rt_2000.csv"


'2000'

In [110]:
df.columns

Index(['Unnamed: 0', 'weekend', 'top10gross', 'top10percentchange',
       'overallgross', 'overallpercentchange', 'numreleases', 'num1release',
       'weeknum', 'RTLink', 'weekendlink', 'num1link'],
      dtype='object')